In [1]:
import numpy as np
import scipy as sp
from numba import jit
import matplotlib.pyplot as plt

## Low level functions
Including: 
- dprod  - for large matrices need Sparse?
- Chebyshev nodes  - different from the numpy version as both endpoints and Lobatto nodes are included
- Monomial and Chebyshev basis matrix
- Monomial and Chebyshev difference operator

Need:
- Splines
- High level extension for multidimensional basis matrices and Chebnodes - easy

In [2]:
@jit
def dprod(A,B):
    nobsa,na= A.shape
    nobsb,nb= B.shape
    Res= np.empty((nobsa,nb*na))
    if (nobsa!=nobsb):
        return 'A and B must have same number of rows'
    for t in range(nobsa):
        for ia in range(na):
            for ib in range(nb):            
                Res[t,nb*(ia-1)+ib]=A[t,ia] * B[t, ib]
    return Res

In [9]:
@jit
def Chebnodes(p, nodetype=0): # Cheb nodes- 1d
    n, a, b = p[0], p[1], p[2]
    s = (b-a) / 2 
    m = (b+a) / 2  
    if (nodetype < 2):  # usual nodes
        k = np.pi*np.linspace(0.5,n-0.5,n)  
        x = m - np.cos(k[0:n]/n) * s  
        if (nodetype == 1):  # Extend nodes to endpoints
            aa = x[0]  
            bb = x[-1]  
            x = (bb*a - aa*b)/(bb-aa) + (b-a)/(bb-aa)*x
    else: # Lobatto nodes
        k = np.pi*np.linspace(0,n-1,n)
        x = m - np.cos(k[0:n]/(n-1)) * s
    return x

In [8]:
@jit
def Chebbasex(p, x): # Cheb basis matrix - 1d
    n, a, b = p[0], p[1], p[2]
    z = (2/(b-a)) * (x-(a+b)/2)
    m = z.shape[0]
    bas = np.empty((m, n));
    bas[:, 0] = 1.0
    bas[:, 1] = z[:]
    z = 2 * z
    for i in range(m):
        for j in range(2,n):
            bas[i, j] = z[i] * bas[i, j-1] - bas[i, j-2]
    return bas

In [7]:
@jit
def Monobasex(p, x):  # Monomials basis matrix- 1d
    n, a, b = p[0], p[1], p[2]
    z = (2/(b-a)) * (x-(a+b)/2)
    m = z.shape[0]
    bas = np.empty((m, n));
    bas[:, 0] = 1.0
    for i in range(m):
        for j in range(1,n):
            bas[i, j] = z[i] * bas[i, j-1]
    return bas

In [33]:
@jit
def ChebDiffMatrix(p):  # Neat code from gregvw/chebyshev-methods/chebdiff.py on github
    n, a, b = p[0], p[1], p[2]  #Differentiating matrix for Chebyshev polynomials
    k= np.arange(n)
    aa= (-1)**k
    A= sp.triu(1-np.outer(aa,aa))
    D= np.dot(A,np.diag(k))
    D[0,:] = D[0,:]/2
    D= D/(b-a)*2
    return D

In [63]:
@jit
def MonoDiffMatrix(p):  
    n, a, b = p[0], p[1], p[2]  #Differentiating matrix for mononomials
    k= np.arange(n)
    D= np.zeros((n,n))
    D[0:(n-1),:]=np.diag(k)[1:n,:]
    D= D/(b-a)*2
    return D

## Test Functions
Examples for: 
- dprod for large matrices
- Approximating cos(x) and it's derivative using monomials and Chebyshev polynomial
- Approximate a multidimensional function

In [6]:
%%timeit
A1=np.random.randn(100,10)
B1=np.random.randn(100,1000)
C1=dprod(A1,B1)

The slowest run took 10.92 times longer than the fastest. This could mean that an intermediate result is being cached 
10 loops, best of 3: 12.5 ms per loop


In [43]:
#Approximate cos(x) with Chebychev
n=20
x=Chebnodes((n,0,20),2)
y= np.cos(x)
Phi=Chebbasex((n,0,20),x)
#Invert to get polynomial coefficients
coeff= np.linalg.inv(Phi) @ y
D=ChebDiffMatrix((n,0,20))
x1=Chebnodes((100,0,20),0)
y1=np.cos(x1)
z1=-np.sin(x1)
Phi1=Chebbasex((n,0,20),x1)
y_approx= Phi1 @ coeff
z_approx= Phi1 @ D @ coeff
plt.plot(x1,z1)
plt.plot(x1,z_approx)
plt.show()

In [65]:
#Approximate cos(x) with monomial
n=20
x=Chebnodes((n,0,20),2)
y= np.cos(x)
Phi=Monobasex((n,0,20),x)
#Invert to get polynomial coefficients
coeff= np.linalg.inv(Phi) @ y
D=MonoDiffMatrix((n,0,20))
x1=Chebnodes((100,0,20),0)
y1=np.cos(x1)
z1=-np.sin(x1)
Phi1=Monobasex((n,0,20),x1)
y_approx= Phi1 @ coeff
z_approx= Phi1 @ D @ coeff
plt.plot(x1,z1)
plt.plot(x1,z_approx)
plt.show()

In [69]:
# Approximate the banana function - to be continued
n= 20
x=Chebnodes((n,-3,3),2)
np.kron(x,np.ones((20,1)))

array([[-3.        , -2.95908391, -2.83745173, -2.63842125, -2.36742153,
        -2.03184471, -1.64084447, -1.20508627, -0.73645646, -0.24773804,
         0.24773804,  0.73645646,  1.20508627,  1.64084447,  2.03184471,
         2.36742153,  2.63842125,  2.83745173,  2.95908391,  3.        ],
       [-3.        , -2.95908391, -2.83745173, -2.63842125, -2.36742153,
        -2.03184471, -1.64084447, -1.20508627, -0.73645646, -0.24773804,
         0.24773804,  0.73645646,  1.20508627,  1.64084447,  2.03184471,
         2.36742153,  2.63842125,  2.83745173,  2.95908391,  3.        ],
       [-3.        , -2.95908391, -2.83745173, -2.63842125, -2.36742153,
        -2.03184471, -1.64084447, -1.20508627, -0.73645646, -0.24773804,
         0.24773804,  0.73645646,  1.20508627,  1.64084447,  2.03184471,
         2.36742153,  2.63842125,  2.83745173,  2.95908391,  3.        ],
       [-3.        , -2.95908391, -2.83745173, -2.63842125, -2.36742153,
        -2.03184471, -1.64084447, -1.20508627, -